In [ ]:
import pandas as pd
import glob
import datetime

In [ ]:
def extract_data(data_in):
    option_type = data_in.columns[-2:]
    data_out = pd.DataFrame(columns=['optiontype', 'date', 'underlying_price', 'strike', 'description', 'delta', 'openinterest', 'total_investment'])
    underlying_price = data_in['underlyingPrice'][0]
    for option in option_type:
        dates = data_in[option].keys()
        for date in dates:
            openinterest = []
            description = []
            delta = []
            strikes = data_in[option][date].keys()
            for strike in strikes:
                openinterest.append(data_in[option][date][strike][0]['openInterest'])
                description.append(data_in[option][date][strike][0]['description'])
                delta.append(data_in[option][date][strike][0]['delta'])
            temp = pd.DataFrame()
            temp['openinterest'] = openinterest
            temp['strike'] = strikes
            temp['date'] = date.split(':')[0]
            temp['optiontype'] = option[:-10]
            temp['description'] = description
            temp['delta'] = delta
            data_out = data_out.append(temp)
    data_out['underlying_price'] = underlying_price
    data_out['total_investment'] = data_out.apply(lambda x: float(x.delta) * float(x.openinterest) * float(x.underlying_price) * 100, axis=1)
    data_out['impact'] = data_out['total_investment'].apply(lambda x: abs(x))
    return data_out

In [ ]:
datapath = './Data'
def get_files(datapath):
    files = glob.glob(datapath+'/*chain*.json')
    files.sort()
    return files

In [ ]:
def get_price_range(file):
    data_in = pd.read_json(file)
    data_out = extract_data(data_in)
    expiration_dates = list(data_out['date'].unique())
    
    price_low = []
    price_high = []
    price_underlying = []

    for date in expiration_dates:
        call_data = data_out[(data_out['date']==date) & (data_out['optiontype']=='call')].sort_values('impact', ascending=False).reset_index(drop=True)
        call_invested = call_data['impact'].sum()
        call_data['impact_percent'] = call_data['impact'].apply(lambda x: x/call_invested)
        call_data['impact_price'] = call_data.apply(lambda x: x.impact_percent*float(x.strike), axis=1)
        price_low.append(call_data['impact_price'].sum())

        put_data = data_out[(data_out['date']==date) & (data_out['optiontype']=='put')].sort_values('impact', ascending=False).reset_index(drop=True)
        put_invested = put_data['impact'].sum()
        put_data['impact_percent'] = put_data['impact'].apply(lambda x: x/put_invested)
        put_data['impact_price'] = put_data.apply(lambda x: x.impact_percent*float(x.strike), axis=1)
        price_high.append(put_data['impact_price'].sum())
        
        price_underlying.append(call_data['underlying_price'].max())

    price_range = pd.DataFrame()
    price_range['date'] = expiration_dates
    price_range['call_thresh']  = price_low
    price_range['put_thresh'] = price_high
    price_range['underlying'] = price_underlying
    return price_range

# Get price range for expiration

In [ ]:
files = get_files(datapath)

In [ ]:
price_range = get_price_range(files[-1])

# Backtesting

In [ ]:
master_df = pd.DataFrame()
for file in files:
    price_range = get_price_range(file)
    master_df = pd.concat([master_df, price_range])

In [ ]:
expirations = master_df['date'].unique()
expirations.sort()

In [ ]:
expirations

In [ ]:
expiration = '2021-07-02'
master_df[(master_df['date']==expiration) & master_df['call_thresh'] != 0.0].reset_index(drop=True).plot(y=['call_thresh', 'put_thresh', 'underlying'], grid=True)

In [ ]:
for expiration in expirations:
    master_df[(master_df['date']==expiration) & master_df['call_thresh'] != 0.0].reset_index(drop=True).plot(y=['call_thresh', 'put_thresh', 'underlying'], title=expiration, grid=True)

# Generate plot

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
fig = plt.figure()
x = price_range['date']
xn = range(len(x))
plt.xticks(xn, x, fontsize=12, rotation=90)
plt.plot(xn, price_range['low'], marker=None, color='red')
plt.plot(xn, price_range['high'], marker=None, color='green')
plt.yticks(np.arange(0.9*min(price_range['low']), 1.1*max(price_range['low']), 100.0))
plt.grid()
import datetime
date_string = datetime.datetime.today().strftime("%Y-%m-%d")
plt.savefig('./Data/{}.png'.format(date_string))

In [ ]:
import base64
import datetime
date_string = datetime.datetime.today().strftime("%Y-%m-%d")
data_uri = base64.b64encode(open('Data/{}.png'.format(date_string), 'rb').read()).decode('utf-8')
img_tag = '<img src="data:image/png;base64,{0}">'.format(data_uri)
print(img_tag)